In [13]:
import requests
from bs4 import BeautifulSoup
import json

base_url = "https://luatvietnam.vn/vi-pham-hanh-chinh/nghi-dinh-100-2019-nd-cp-xu-phat-vi-pham-giao-thong-179619-d1.html"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
}

response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

chapters = []
current_chapter = None
current_text = ''

docitems = soup.find_all("div", class_=lambda x: x and x.startswith("docitem"))
docitems = docitems[4:]  # Skip header elements

for div in docitems:
    class_name = div.get('class')[0]
    
    if class_name == 'docitem-2':
        # Save previous chapter's content
        if current_chapter and current_text:
            current_chapter['content'].append(current_text)
        
        # Start new chapter
        if current_chapter:
            chapters.append(current_chapter)
        current_chapter = {
            'title': div.text.strip(),
            'content': []
        }
        current_text = ''
        
    elif class_name == 'docitem-5':
        # Save previous section if exists
        if current_text:
            current_chapter['content'].append(current_text)
        # Start new section
        current_text = div.text.strip()
        
    elif class_name in ['docitem-11', 'docitem-12']:
        # Append to current section
        current_text += '\n' + div.text.strip()

# Save final chapter and its content
if current_chapter and current_text:
    current_chapter['content'].append(current_text)
if current_chapter:
    chapters.append(current_chapter)

# Save to JSON file
with open('chapters.json', 'w', encoding='utf-8') as f:
    json.dump(chapters, f, ensure_ascii=False, indent=2)

print(f"Saved {len(chapters)} chapters to chapters.json")

Saved 5 chapters to chapters.json
